# Интро и полезная литература

### Литература

- [Word2vec в картинках](https://habr.com/ru/post/446530/). Статья на хабре от 2019 года.
- [Презентация](http://www.machinelearning.ru/wiki/images/b/b3/Word2Vec.pdf) (2017 год) с примерами реализации кода на библиотеке gensim
 
Так как Word2Vec - это уже не просто единичная реализация, а группа алгоритмов для получения векторных представлений слов, то существует масса различных пакетов на эту тему. Мы будем пользоваться gensim.

- [Gensim – Руководство для начинающих](https://webdevblog.ru/gensim-rukovodstvo-dlya-nachinajushhih/). Здесь нужно изучить ряд терминов, чтобы понимать принцип работы библиотеки. Статья 2019 года.
- [A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model](https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92). Статья 2019 года. По ней хорошо учиться писать код.

### Термины

- **Токен** обычно означает «слово»
- **Документ** обычно может относиться как к «предложению» так и к «абзацу»
- **Корпус** обычно представляет собой «собрание документов» в виде пакета слов или как его еще называют мешка слов 

# Импорт локальных констант и функций

Вначале перейдём на уровень рутового пути репозитория
(в дальнейшем всегда пишем пути от корня репозитория)

In [1]:
cd ..

/Users/polina/WebstormProjects/ml-bug-detector


Заимпортируем всё, что нам пригодится в дальнейшем

In [2]:
from shared.helpers.word2VecPreprocessing import filter_tokens_fo_Word2Vec

Также внешние библиотеки подключим сразу

In [3]:
import pandas as pd
import json
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count

# Строим модель

In [4]:
pathToTokenizeScriptsJson = 'data/parsed-scripts.json'
  
tokenizedScriptsJson = open(pathToTokenizeScriptsJson) 
tokenized_scripts_dictionary = json.load(tokenizedScriptsJson)

In [5]:
for fileName in tokenized_scripts_dictionary.keys():
    tokenized_scripts_dictionary[fileName] = filter_tokens_fo_Word2Vec(tokenized_scripts_dictionary[fileName])

In [6]:
tokenized_scripts_list_of_lists = list(tokenized_scripts_dictionary.values())

Теперь бы нужно настроить гиперпараметры.

[Эта статья](https://habr.com/ru/post/446530/) говорит, что embedding_size устанавливают обычно на 300.

In [7]:
word2VecModel = Word2Vec(
    # list of lists. Например: [["cat", "say", "meow"], ["dog", "say", "woof"]]
    sentences=tokenized_scripts_list_of_lists,
    
    # The number of dimensions of the embeddings and the default is 100
    size=300, 
    
    #The maximum distance between a target word and words around the target word. The default window is 5.
    window=5,
    
    # финальный размер словаря
    max_final_vocab=10000, 
    
    # игнорировать слова с частотностью ниже, чем эта (регулируется автоматически, если установлено max_final_vocab)
    min_count = 5, 
    
    workers=cpu_count(),
    
    # The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW
    sg=0
)

# Сохранение и загрузка модели

In [8]:
word2VecModel.save('word2Vec/word2VecModel')

In [9]:
from gensim.models.word2vec import Word2Vec
word2VecModel = Word2Vec.load('word2Vec/word2VecModel')

In [10]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

[('timeout', 0.9540337324142456),
 ('clearTimeout', 0.9249351024627686),
 ('close', 0.9224143624305725),
 ('sender', 0.9207724332809448),
 ('attach', 0.9160687923431396),
 ('on', 0.9150377511978149),
 ('onUpdate', 0.9130774736404419),
 ('addEventListener', 0.910114049911499),
 ('evt', 0.8991033434867859),
 ('off', 0.8965657353401184),
 ('that', 0.890986442565918),
 ('_self', 0.8894838690757751),
 ('timer', 0.8801249265670776),
 ("'click'", 0.8786412477493286),
 ('deferred', 0.8764210343360901),
 ('$timeout', 0.8728090524673462),
 ('send', 0.8708072900772095),
 ('controller', 0.868100643157959),
 ('unbind', 0.8631912469863892),
 ('render', 0.8626474142074585)]

Таки да.

- clearTimeout - обрывает запланированный вызов функции
- timeout - через какое время исполнить функцию (один из агрументов всегда)
- cb - это callback-функции (один из агрументов всегда)

In [11]:
print('В нашем словаре теперь', len(word2VecModel.wv.vocab), 'слов')

В нашем словаре теперь 8896 слов
